I utilized the SVM implementation specifically tailored for EEG data from the following GitHub repository: https://github.com/jayavardhanravi/EEG-Data-predection/blob/master/mypart1.py. This resource provided invaluable support for my SVM analysis.

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
ec_data_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC"
eo_data_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EO"
ec_eeg_data = np.load(os.path.join(ec_data_dir, "normalized_epoch_eeg_data.npy"))
eo_eeg_data = np.load(os.path.join(eo_data_dir, "normalized_epoch_eeg_data.npy"))

In [4]:
print(ec_eeg_data.shape)
print(eo_eeg_data.shape)

(4356, 1, 32, 4975)
(4344, 1, 32, 4975)


In [5]:
ec_labels_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EC"
eo_labels_dir = "/content/drive/MyDrive/TD-BRAIN/training_data/data/EO"
ec_eeg_labels = np.load(os.path.join(ec_labels_dir, "labels_data.npy"))
eo_eeg_labels = np.load(os.path.join(eo_labels_dir, "labels_data.npy"))

In [6]:
print(ec_eeg_labels.shape)
print(eo_eeg_labels.shape)

(4356, 2)
(4344, 2)


In [7]:
for label in ec_eeg_labels:
  sample_id = label[0]
  if sample_id not in eo_eeg_labels[:, 0]:
        index_to_remove = np.where(ec_eeg_labels[:, 0] == sample_id)[0]
        ec_eeg_labels = np.delete(ec_eeg_labels, index_to_remove, axis=0)
        ec_eeg_data = np.delete(ec_eeg_data, index_to_remove, axis=0)
print(ec_eeg_labels.shape)
print(ec_eeg_data.shape)

eeg_data = np.concatenate((ec_eeg_data[:, 0], eo_eeg_data[:, 0]), axis=1)
eeg_data.shape

eeg_labels = ec_eeg_labels

(4344, 2)
(4344, 1, 32, 4975)


In [8]:
healthy_count, mdd_count = 0, 0
for sample in eeg_labels:
  if sample[1] == "MDD":
      mdd_count += 1
  else:
      healthy_count += 1

print(f"Number of MDD participants: {mdd_count}")
print(f"Number of Healthy participants: {healthy_count}")

Number of MDD participants: 3780
Number of Healthy participants: 564


#Extracting data for female participants

In [9]:
df_participants = pd.read_pickle('/content/drive/MyDrive/TD-BRAIN/TDBRAIN_participants_V2_data/df_participants.pkl')
female_count = 0

eeg_data_female = []
eeg_label_female = []

for i, labels in enumerate(eeg_labels):
     sample_id = labels[0]
     index = df_participants.loc[df_participants['participants_ID'] == sample_id].index
     if [value for value in df_participants.loc[index, 'gender']][0] == 1:
       eeg_data_female.append(eeg_data[i])
       eeg_label_female.append(eeg_labels[i])

eeg_data_female = np.array(eeg_data_female)
eeg_label_female = np.array(eeg_label_female)

print(eeg_data_female.shape, eeg_label_female.shape)

(1932, 64, 4975) (1932, 2)


In [10]:
healthy_count_female, mdd_count_female = 0, 0
for sample in eeg_label_female:
  if sample[1] == "MDD":
      mdd_count_female += 1
  else:
      healthy_count_female += 1

print(f"Number of MDD female participants: {mdd_count_female}")
print(f"Number of Healthy female participants: {healthy_count_female}")

Number of MDD female participants: 1740
Number of Healthy female participants: 192


# **Model**

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils import resample, shuffle

In [12]:
ll = eeg_label_female
encountered_sample_ids = {}
sample_ids_with_more_than_12_entries = []

for index, sample_id in enumerate(ll):
    sample_id_tuple = tuple(sample_id)
    count = encountered_sample_ids.get(sample_id_tuple, 0)
    count += 1
    encountered_sample_ids[sample_id_tuple] = count
    if count > 12:
        sample_ids_with_more_than_12_entries.append((sample_id_tuple, index))

indices_to_remove = [index for _, index in sample_ids_with_more_than_12_entries]
eeg_label_female = [sample for i, sample in enumerate(eeg_label_female) if i not in indices_to_remove]
eeg_data_female = [data for i, data in enumerate(eeg_data_female) if i not in indices_to_remove]
print("Length of filtered eeg_label_female:", len(eeg_label_female))
print("Length of filtered eeg_data_female:", len(eeg_data_female))


###### Undersampling and preparing training data ########
ll = eeg_label_female
unique_sample_id = []
encountered_sample_ids = set()
print(len(ll))
for sample_id in ll:
    sample_id_tuple = tuple(sample_id)
    if sample_id_tuple not in encountered_sample_ids:
        unique_sample_id.append(sample_id)
        encountered_sample_ids.add(sample_id_tuple)
print(len(unique_sample_id))

num_samples_minority = 16
indices_maj = [index for index, sample in enumerate(unique_sample_id) if sample[1] == "MDD"]
indices_min = [index for index, sample in enumerate(unique_sample_id) if sample[1] == "HEALTHY"]
undersampled = np.random.choice(indices_maj, num_samples_minority, replace=False)

balanced_data_indices = np.concatenate([indices_min, undersampled])
# print(unique_sample_id)
balanced_unique_sample_id = [unique_sample_id[i] for i in balanced_data_indices]

# Extract all unique sample IDs from train_unique_sample_id
unique_sample_ids = [sample_id[0] for sample_id in balanced_unique_sample_id]
print(len(unique_sample_ids))
# Extract all indices from eeg_labels for sample IDs in train_unique_sample_id
indices = []
for i, sample_id in enumerate(eeg_label_female):
  # print(sample_id[0])
  if sample_id[0] in unique_sample_ids:
        indices.append(i)

# Convert indices to a NumPy array
indices = np.array(indices)
X_train = []
y_train = []
for i in indices:
    X_train.append(eeg_data_female[i])
    y_train.append(eeg_label_female[i])

X_train = np.array(X_train)
y_train = np.array(y_train)

# Shuffle together with their indices
permutation = np.random.permutation(len(X_train))
X_train = X_train[permutation]
y_train = y_train[permutation]

print(X_train.shape)
# print(y_train)

sample_ids = []
for sample in y_train:
  sample_ids.append(sample[0])
sample_ids = np.array(sample_ids)
l = np.array([1 if label[1] == "MDD" else 0 for label in y_train])

Length of filtered eeg_label_female: 1920
Length of filtered eeg_data_female: 1920
1920
160
32
(384, 64, 4975)


# Results

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_class_weight

# Data preparation steps
ll = eeg_label_female
encountered_sample_ids = {}
sample_ids_with_more_than_12_entries = []

for index, sample_id in enumerate(ll):
    sample_id_tuple = tuple(sample_id)
    count = encountered_sample_ids.get(sample_id_tuple, 0)
    count += 1
    encountered_sample_ids[sample_id_tuple] = count
    if count > 12:
        sample_ids_with_more_than_12_entries.append((sample_id_tuple, index))

indices_to_remove = [index for _, index in sample_ids_with_more_than_12_entries]
eeg_label_female = [sample for i, sample in enumerate(eeg_label_female) if i not in indices_to_remove]
eeg_data_female = [data for i, data in enumerate(eeg_data_female) if i not in indices_to_remove]
print("Length of filtered eeg_label_female:", len(eeg_label_female))
print("Length of filtered eeg_data_female:", len(eeg_data_female))

###### Undersampling and preparing training data ########
ll = eeg_label_female
unique_sample_id = []
encountered_sample_ids = set()
print(len(ll))
for sample_id in ll:
    sample_id_tuple = tuple(sample_id)
    if sample_id_tuple not in encountered_sample_ids:
        unique_sample_id.append(sample_id)
        encountered_sample_ids.add(sample_id_tuple)
print(len(unique_sample_id))

num_samples_minority = 16
indices_maj = [index for index, sample in enumerate(unique_sample_id) if sample[1] == "MDD"]
indices_min = [index for index, sample in enumerate(unique_sample_id) if sample[1] == "HEALTHY"]
undersampled = np.random.choice(indices_maj, num_samples_minority, replace=False)

balanced_data_indices = np.concatenate([indices_min, undersampled])
balanced_unique_sample_id = [unique_sample_id[i] for i in balanced_data_indices]

# Extract all unique sample IDs from balanced_unique_sample_id
unique_sample_ids = [sample_id[0] for sample_id in balanced_unique_sample_id]
print(len(unique_sample_ids))
# Extract all indices from eeg_label_female for sample IDs in balanced_unique_sample_id
indices = []
for i, sample_id in enumerate(eeg_label_female):
    if sample_id[0] in unique_sample_ids:
        indices.append(i)

# Convert indices to a NumPy array
indices = np.array(indices)
X = np.array([eeg_data_female[i] for i in indices])
y = np.array([1 if label[1] == "MDD" else 0 for label in eeg_label_female if label[0] in unique_sample_ids])

# Shuffle together with their indices
permutation = np.random.permutation(len(X))
X = X[permutation]
y = y[permutation]

print(X.shape)

# Define the EEGClassifier class
class EEGClassifier:
    def __init__(self):
        self.model = SVC(C=0.4, kernel='rbf')

    def train(self, X_train, y_train):
        X_train_flattened = X_train.reshape(X_train.shape[0], -1)
        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = dict(zip(np.unique(y_train), class_weights))
        self.model = SVC(C=0.4, kernel='rbf', class_weight=class_weight_dict)
        self.model.fit(X_train_flattened, y_train)

    def evaluate(self, X_test, y_test):
        X_test_flattened = X_test.reshape(X_test.shape[0], -1)
        y_pred = self.model.predict(X_test_flattened)
        return {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, average='macro'),
            'recall': recall_score(y_test, y_pred, average='macro'),
            'f1_score': f1_score(y_test, y_pred, average='macro'),
            'confusion_matrix': confusion_matrix(y_test, y_pred)
        }

def main():
    global X, y
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    classifier = EEGClassifier()

    num_splits = 5
    cv = KFold(n_splits=num_splits, shuffle=True, random_state=42)

    val_accuracies = []
    val_precisions = []
    val_recalls = []
    val_f1_scores = []
    val_confusion_matrices = []

    for fold_idx, (train_index, val_index) in enumerate(cv.split(X_train), 1):
        print(f"Fold {fold_idx}:")

        X_train_fold = X_train[train_index]
        y_train_fold = y_train[train_index]
        X_val_fold = X_train[val_index]
        y_val_fold = y_train[val_index]

        classifier.train(X_train_fold, y_train_fold)

        train_metrics = classifier.evaluate(X_train_fold, y_train_fold)
        print(f'Training Results - Accuracy: {train_metrics["accuracy"]}, Precision: {train_metrics["precision"]}, '
              f'Recall: {train_metrics["recall"]}, F1 Score: {train_metrics["f1_score"]}')

        val_metrics = classifier.evaluate(X_val_fold, y_val_fold)
        print(f'Validation Results - Accuracy: {val_metrics["accuracy"]}, Precision: {val_metrics["precision"]}, '
              f'Recall: {val_metrics["recall"]}, F1 Score: {val_metrics["f1_score"]}')
        print('Validation Confusion Matrix:')
        print(val_metrics['confusion_matrix'])
        print()

        val_accuracies.append(val_metrics["accuracy"])
        val_precisions.append(val_metrics["precision"])
        val_recalls.append(val_metrics["recall"])
        val_f1_scores.append(val_metrics["f1_score"])
        val_confusion_matrices.append(val_metrics["confusion_matrix"])

    # Calculate average validation metrics
    avg_val_accuracy = np.mean(val_accuracies)
    avg_val_precision = np.mean(val_precisions)
    avg_val_recall = np.mean(val_recalls)
    avg_val_f1_score = np.mean(val_f1_scores)
    avg_val_confusion_matrix = np.mean(val_confusion_matrices, axis=0)

    print(f'Average Validation Accuracy: {avg_val_accuracy}')
    print(f'Average Validation Precision: {avg_val_precision}')
    print(f'Average Validation Recall: {avg_val_recall}')
    print(f'Average Validation F1 Score: {avg_val_f1_score}')
    print(f'Average Validation Confusion Matrix: \n{avg_val_confusion_matrix}')

    # Evaluate on the test set
    test_metrics = classifier.evaluate(X_test, y_test)
    print(f'Test Results - Accuracy: {test_metrics["accuracy"]}, Precision: {test_metrics["precision"]}, '
          f'Recall: {test_metrics["recall"]}, F1 Score: {test_metrics["f1_score"]}')
    print('Test Confusion Matrix:')
    print(test_metrics['confusion_matrix'])

if __name__ == "__main__":
    main()


Length of filtered eeg_label_female: 1920
Length of filtered eeg_data_female: 1920
1920
160
32
(384, 64, 4975)
Fold 1:
Training Results - Accuracy: 0.758957654723127, Precision: 0.7720868409393, Recall: 0.7606979113601631, F1 Score: 0.7567874272010962
Validation Results - Accuracy: 0.4675324675324675, Precision: 0.44565217391304346, Recall: 0.45426829268292684, F1 Score: 0.43675289919714544
Validation Confusion Matrix:
[[ 16 16]
 [ 17 13]]

Fold 2:
Training Results - Accuracy: 0.739413680781759, Precision: 0.8119015047879616, Recall: 0.7523627075351214, F1 Score: 0.7294200599330161


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Results - Accuracy: 0.6493506493506493, Precision: 0.6274509803921569, Recall: 0.5921985815602837, F1 Score: 0.586597733147743
Validation Confusion Matrix:
[[19  13]
 [9  22]]

Fold 3:
Training Results - Accuracy: 0.7947882736156352, Precision: 0.8139415340604327, Recall: 0.7983391385608699, F1 Score: 0.7928097783586325
Validation Results - Accuracy: 0.5844155844155844, Precision: 0.5731523378582202, Recall: 0.5663474692202461, F1 Score: 0.5631205673758866
Validation Confusion Matrix:
[[16  15]
 [11  20]]

Fold 4:
Training Results - Accuracy: 0.7850162866449512, Precision: 0.8046134899583175, Recall: 0.7744771660264618, F1 Score: 0.7761797454931071


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Results - Accuracy: 0.4805194805194805, Precision: 0.5545454545454546, Recall: 0.5488888888888889, F1 Score: 0.478319783197832
Validation Confusion Matrix:
[[11  15]
 [17  18]]

Fold 5:
Training Results - Accuracy: 0.7597402597402597, Precision: 0.7767857142857143, Recall: 0.7563713080168777, F1 Score: 0.754258851942899
Validation Results - Accuracy: 0.4342105263157895, Precision: 0.44588235294117645, Recall: 0.45168067226890757, F1 Score: 0.4261633011413521
Validation Confusion Matrix:
[[14 15]
 [19 12]]

Average Validation Accuracy: 0.5232901639344261
Average Validation Precision: 0.529764857983417
Average Validation Recall: 0.5223240753314168
Average Validation F1 Score: 0.49828766354080567
Average Validation Confusion Matrix: 
[[15.2 14.8]
 [14.6 17]]
Test Results - Accuracy: 0.5857142857142855, Precision: 0.4284750337381916, Recall: 0.58584750337381916, F1 Score: 0.5684750337381916
Test Confusion Matrix:
[[22 16]
 [18 21]]
